In [38]:
from constants import SDK_PATH, DATA_PATH, WORD_EMB_PATH, CACHE_PATH
import sys
import pandas as pd

In [39]:


if SDK_PATH is None:
    print("SDK path is not specified! Please specify first in constants/paths.py")
    exit(0)
else:
    sys.path.append(SDK_PATH)
    print(f"SDK path is set to {SDK_PATH}")

SDK path is set to C:\Users\Viki\Documents\Thesis\CMU-MultimodalSDK


In [40]:
import sys
for path in sys.path:
    print(path)
    

C:\Program Files\JetBrains\PyCharm 2023.3.2\plugins\python\helpers-pro\jupyter_debug
C:\Program Files\JetBrains\PyCharm 2023.3.2\plugins\python\helpers\pydev
C:\Users\Viki\Documents\Thesis\tryout3
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\python310.zip
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\DLLs
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib
C:\Users\Viki\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0
C:\Users\Viki\Documents\Thesis\tryout3\.venv

C:\Users\Viki\Documents\Thesis\tryout3\.venv\lib\site-packages
C:\Users\Viki\Documents\Thesis\tryout3\.venv\lib\site-packages\win32
C:\Users\Viki\Documents\Thesis\tryout3\.venv\lib\site-packages\win32\lib
C:\Users\Viki\Documents\Thesis\tryout3\.venv\lib\site-packages\Pythonwin
C:\Users\Viki\Documents\Thesis\CMU-MultimodalSDK
C:\Users\Viki\D

In [41]:
import mmsdk
import os
import re
import numpy as np
from mmsdk import mmdatasdk as md
from subprocess import check_call, CalledProcessError

import pandas as pd
import csv


# we can see they are in the format of 'video_id[segment_no]', but the splits was specified with video_id only
# we need to use regex or something to match the video IDs...
import torch
import torch.nn as nn

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm_notebook
from collections import defaultdict
import random

from torch.optim import Adam, SGD
from sklearn.metrics import accuracy_score
import tqdm
from tqdm import tqdm_notebook
import random
import numpy as np


from torch import optim
from torch.utils.tensorboard import SummaryWriter





In [42]:

if not os.path.exists(DATA_PATH):
    check_call(' '.join(['mkdir', '-p', DATA_PATH]), shell=True)

DATASET = md.cmu_mosi

try:
    md.mmdataset(DATASET.highlevel, DATA_PATH)
except RuntimeError:
    print("High-level features have been downloaded previously.")

try:
    md.mmdataset(DATASET.raw, DATA_PATH)
except RuntimeError:
    print("Raw data have been downloaded previously.")
    
try:
    md.mmdataset(DATASET.labels, DATA_PATH)
except RuntimeError:
    print("Labels have been downloaded previously.")

Normalized destination path: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu/CMU-MOSI/language/CMU_MOSI_TimestampedWordVectors.csd
[2024-12-09 12:23:29.571] | Error   | C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu/CMU-MOSI/language/CMU_MOSI_TimestampedWordVectors.csd file already exists ...
High-level features have been downloaded previously.
Normalized destination path: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu/CMU-MOSI/language/CMU_MOSI_TimestampedWords.csd
[2024-12-09 12:23:29.574] | Error   | C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu/CMU-MOSI/language/CMU_MOSI_TimestampedWords.csd file already exists ...
Raw data have been downloaded previously.
Normalized destination path: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu/CMU-MOSI/labels/CMU_MOSI_Opinion_Labels.csd
[2024-12-09 12:23:29.574] | Error   | C:\Users

In [43]:
# list the directory contents... let's see what features there are
data_files = os.listdir(DATA_PATH)
print('\n'.join(data_files))

embedding_and_mapping.pt
http__immortal.multicomp.cs.cmu.edu


In [44]:

folder_path = "C:\\Users\\Viki\\Documents\\Thesis\\tryout3\\data"
for root, dirs, files in os.walk(folder_path):
    print(f"Directory: {root}")
    for file in files:
        print(f"  {file}")


Directory: C:\Users\Viki\Documents\Thesis\tryout3\data
  embedding_and_mapping.pt
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\acoustic
  CMU_MOSI_COVAREP.csd
  CMU_MOSI_OpenSmile_EB10.csd
  CMU_MOSI_openSMILE_IS09.csd
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\labels
  CMU_MOSI_Opinion_Labels.csd
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\language
  CMU_MOSI_TimestampedPhones.csd
  CMU_MOSI_TimestampedWords.csd
  CMU_MOSI_TimestampedWordVectors.csd
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\visual
  CMU_MOSI_Visual_Facet_41.csd
  CMU_MOSI_Visual_Facet_42.csd
  CMU_MOSI_Visual

In [45]:
# Define your different modalities - refer to the filenames of the CSD files
visual_field_Facet41 = 'CMU_MOSI_Visual_Facet_41'
visual_field_Facet42 = 'CMU_MOSI_Visual_Facet_42'
visual_field_OpenFace1 = 'CMU_MOSI_Visual_OpenFace_1'


# visual_field_OpenFace2 = 'CMU_MOSI_Visual_OpenFace_2'
# [2024-11-24 21:59:03.886] | Error   | C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\visual\CMU_MOSI_Visual_OpenFace_2.csd resource is not a valid hdf5 computational sequence format ...


acoustic_field_COVAREP = 'CMU_MOSI_COVAREP'
acoustic_field_OpenSmile_EB10 = 'CMU_MOSI_OpenSmile_EB10'
acoustic_field_OpenSmile_IS09 = 'CMU_MOSI_openSMILE_IS09'



text_field_Words = 'CMU_MOSI_TimestampedWords'
text_field_Phones = 'CMU_MOSI_TimestampedPhones'
text_field_WordVectors = 'CMU_MOSI_TimestampedWordVectors'




In [46]:

# text_field = 'CMU_MOSI_ModifiedTimestampedWords'


# List of features
features = [
    text_field_Words,
    # text_field_Phones,
    text_field_WordVectors,
    visual_field_Facet41, 
    # visual_field_Facet42,
    # visual_field_OpenFace1,
    # visual_field_OpenFace2,
    acoustic_field_COVAREP,
    # acoustic_field_OpenSmile_IS09,
    # acoustic_field_OpenSmile_EB10
]


# Recipe with correct subdirectory paths for each modality
recipe = {
    text_field_Words: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "language", text_field_Words) + '.csd',
    
    # not helpful
    # text_field_Phones: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu","CMU-MOSI", "language", text_field_Phones) + '.csd',

    text_field_WordVectors: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "language", text_field_WordVectors) + '.csd',
    
    visual_field_Facet41: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "visual", visual_field_Facet41) + '.csd',
    
    # visual_field_Facet42: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "visual", visual_field_Facet42) + '.csd',
    
    # visual_field_OpenFace1: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "visual", visual_field_OpenFace1) + '.csd',
    
    
    #not taken into account cause not runnable - 
    # visual_field_OpenFace2: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "visual", visual_field_OpenFace2) + '.csd',
    
    acoustic_field_COVAREP: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "acoustic", acoustic_field_COVAREP) + '.csd',
    
    # acoustic_field_OpenSmile_EB10: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "acoustic", acoustic_field_OpenSmile_EB10) + '.csd',
    
    # has same and less features as EB10
    # acoustic_field_OpenSmile_IS09: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "acoustic", acoustic_field_OpenSmile_IS09) + '.csd'
}



print (recipe)
dataset = md.mmdataset(recipe)

{'CMU_MOSI_TimestampedWords': 'C:\\Users\\Viki\\Documents\\Thesis\\tryout3\\data\\http__immortal.multicomp.cs.cmu.edu\\CMU-MOSI\\language\\CMU_MOSI_TimestampedWords.csd', 'CMU_MOSI_TimestampedWordVectors': 'C:\\Users\\Viki\\Documents\\Thesis\\tryout3\\data\\http__immortal.multicomp.cs.cmu.edu\\CMU-MOSI\\language\\CMU_MOSI_TimestampedWordVectors.csd', 'CMU_MOSI_Visual_Facet_41': 'C:\\Users\\Viki\\Documents\\Thesis\\tryout3\\data\\http__immortal.multicomp.cs.cmu.edu\\CMU-MOSI\\visual\\CMU_MOSI_Visual_Facet_41.csd', 'CMU_MOSI_COVAREP': 'C:\\Users\\Viki\\Documents\\Thesis\\tryout3\\data\\http__immortal.multicomp.cs.cmu.edu\\CMU-MOSI\\acoustic\\CMU_MOSI_COVAREP.csd'}
[2024-12-09 12:23:29.959] | Success | Computational sequence read from file C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\language\CMU_MOSI_TimestampedWords.csd ...
[2024-12-09 12:23:29.997] | Status  | Checking the integrity of the <words> computational sequence ...
[2024-12-09 12:23:

[2024-12-09 12:23:30.107] | Success | <words> computational sequence data in correct format.
[2024-12-09 12:23:30.108] | Status  | Checking the format of the metadata in <words> computational sequence ...
[2024-12-09 12:23:30.108] | Warning | <words> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:23:30.110] | Success | Computational sequence read from file C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\language\CMU_MOSI_TimestampedWordVectors.csd ...
[2024-12-09 12:23:30.139] | Status  | Checking the integrity of the <glove_vectors> computational sequence ...
[2024-12-09 12:23:30.139] | Status  | Checking the format of the data in <glove_vectors> computational sequence ...


[2024-12-09 12:23:30.268] | Success | <glove_vectors> computational sequence data in correct format.
[2024-12-09 12:23:30.268] | Status  | Checking the format of the metadata in <glove_vectors> computational sequence ...
[2024-12-09 12:23:30.268] | Warning | <glove_vectors> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:23:30.272] | Success | Computational sequence read from file C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\visual\CMU_MOSI_Visual_Facet_41.csd ...
[2024-12-09 12:23:30.311] | Status  | Checking the integrity of the <FACET_4.1> computational sequence ...
[2024-12-09 12:23:30.311] | Status  | Checking the format of the data in <FACET_4.1> computational sequence ...


[2024-12-09 12:23:30.445] | Success | <FACET_4.1> computational sequence data in correct format.
[2024-12-09 12:23:30.446] | Status  | Checking the format of the metadata in <FACET_4.1> computational sequence ...
[2024-12-09 12:23:30.446] | Warning | <FACET_4.1> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:23:30.448] | Success | Computational sequence read from file C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\acoustic\CMU_MOSI_COVAREP.csd ...
[2024-12-09 12:23:30.479] | Status  | Checking the integrity of the <COVAREP> computational sequence ...
[2024-12-09 12:23:30.479] | Status  | Checking the format of the data in <COVAREP> computational sequence ...


[2024-12-09 12:23:30.601] | Success | <COVAREP> computational sequence data in correct format.
[2024-12-09 12:23:30.601] | Status  | Checking the format of the metadata in <COVAREP> computational sequence ...
[2024-12-09 12:23:30.601] | Warning | <COVAREP> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:23:30.601] | Success | Dataset initialized successfully ... 


In [47]:

print("=== Metadata FACE 41 Visual===\n")
print("Visual FACE 41 Metadata:", dataset[visual_field_Facet41].metadata)
print("\n")

print("=== Metadata COVAREP Acoustic===\n")
print("Acoustic COVAREP Metadata:", dataset[acoustic_field_COVAREP].metadata)
print("\n")

print("=== Metadata Words===\n")
print("Words Metadata:", dataset[text_field_Words].metadata)
print("\n")

print("=== Metadata WordVectors===\n")
print("WordVectors Metadata:", dataset[text_field_WordVectors].metadata)
print("\n")


=== Metadata FACE 41 Visual===

Visual FACE 41 Metadata: {'alignment compatible': True, 'computational sequence description': 'FACET 4.1 Visual Features for CMU-MOSI Dataset', 'computational sequence version': 1.0, 'contact': 'abagherz@andrew.cmu.edu', 'creator': 'Amir Zadeh', 'dataset bib citation': '@article{zadeh2016multimodal,title={Multimodal sentiment intensity analysis in videos: Facial gestures and verbal messages},author={Zadeh, Amir and Zellers, Rowan and Pincus, Eli and Morency, Louis-Philippe},journal={IEEE Intelligent Systems},volume={31},number={6},pages={82--88},year={2016},publisher={IEEE}}', 'dataset name': 'CMU-MOSI', 'dataset version': 1.0, 'dimension names': ['Face X', 'Face Y', 'Face Width', 'Face Height', 'angerEvidence', 'contemptEvidence', 'disgustEvidence', 'joyEvidence', 'fearEvidence', 'negativeEvidence', 'neutralEvidence', 'positiveEvidence', 'sadnessEvidence', 'surpriseEvidence', 'confusionEvidence', 'frustrationEvidence', 'angerIntensity', 'contemptIntensi

In [48]:
fields_metadata = [
    {"Field": "Visual FACE 41", "Metadata": dataset[visual_field_Facet41].metadata},
    # {"Field": "Visual FACE 42", "Metadata": dataset[visual_field_Facet42].metadata},
    # {"Field": "Visual OpenFace", "Metadata": dataset[visual_field_OpenFace1].metadata},
    {"Field": "Acoustic COVAREP", "Metadata": dataset[acoustic_field_COVAREP].metadata},
    # {"Field": "Acoustic OpenSmile EB10", "Metadata": dataset[acoustic_field_OpenSmile_EB10].metadata},
    # {"Field": "Acoustic OpenSmile IS09", "Metadata": dataset[acoustic_field_OpenSmile_IS09].metadata},
    {"Field": "Words", "Metadata": dataset[text_field_Words].metadata},
    # {"Field": "Phones", "Metadata": dataset[text_field_Phones].metadata},
    {"Field": "WordVectors", "Metadata": dataset[text_field_WordVectors].metadata},
]

# Normalize (flatten) the metadata dictionaries
normalized_data = []
for entry in fields_metadata:
    metadata_flat = pd.json_normalize(entry["Metadata"], sep="_")
    metadata_flat["Field"] = entry["Field"]
    normalized_data.append(metadata_flat)

# print(f"normalized metadata: {normalized_data}")

df_metadata = pd.concat(normalized_data, ignore_index=True)
df_metadata = df_metadata[["Field"] + [col for col in df_metadata.columns if col != "Field"]]

print(df_metadata)



# # Save the DataFrame to a CSV file
# csv_filename = "metadata_summary.csv"
# df_metadata.to_csv(csv_filename, index=False)
# 
# print(f"Metadata successfully saved to {csv_filename}")
# 
# 
# 
#remove \ for Acoustic OpenSmile EB10
#remove b" and " for Acoustic OpenSmile IS09

              Field  alignment compatible  \
0    Visual FACE 41                  True   
1  Acoustic COVAREP                  True   
2             Words                 False   
3       WordVectors                  True   

               computational sequence description  \
0  FACET 4.1 Visual Features for CMU-MOSI Dataset   
1  COVAREP Acoustic Features for CMU-MOSI Dataset   
2             Word sequences for CMU-MOSI Dataset   
3      Word vector sequences for CMU-MOSI Dataset   

   computational sequence version                  contact     creator  \
0                             1.0  abagherz@andrew.cmu.edu  Amir Zadeh   
1                             1.0  abagherz@andrew.cmu.edu  Amir Zadeh   
2                             1.0  abagherz@andrew.cmu.edu  Amir Zadeh   
3                             1.0  abagherz@andrew.cmu.edu  Amir Zadeh   

                                dataset bib citation dataset name  \
0  @article{zadeh2016multimodal,title={Multimodal...     CMU-MOSI   

In [49]:
df_components = df_metadata[["Field", "dimension names"]]

# Define the replacements in a dictionary
replacements = {
    'b"': '',
    '"': '',
    '\'' : '',
    '[': '',
    ']': ''

}



def replace_strings(x):
    if isinstance(x, str):
        for old, new in replacements.items():
            x = x.replace(old, new)
    return x


# Apply the replacements using map
df_components = df_components.applymap(replace_strings)

print(df_components)

# Save the DataFrame to a CSV file
csv_filename = "componentsWithoutIS.csv"
df_components.to_csv(csv_filename, index=False)

              Field                                    dimension names
0    Visual FACE 41  [Face X, Face Y, Face Width, Face Height, ange...
1  Acoustic COVAREP  [F0, VUV, NAQ, QOQ, H1H2, PSP, MDQ, peakSlope,...
2             Words                                             [word]
3       WordVectors  [glove_vector, glove_vector, glove_vector, glo...


C:\Users\Viki\AppData\Local\Temp\ipykernel_28880\4218429857.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_components = df_components.applymap(replace_strings)


In [50]:

# Define the replacements in a dictionary
replacements = {
    'b"': '',
    '\"': '',
    '\'': '',
    '[': '',
    ']': ''
}

# Function to apply replacements
def replace_strings(x):
    if isinstance(x, str):
        for old, new in replacements.items():
            x = x.replace(old, new)
    return x

# Read the CSV file into a DataFrame
with open('componentsWithoutIS.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    rows = []
    for row in reader:
        new_row = []
        for element in row:
            # Split elements by comma and strip spaces
            split_elements = [e.strip() for e in element.split(',')]
            new_row.extend(split_elements)
        rows.append(new_row)

# Create DataFrame from the processed rows
df = pd.DataFrame(rows)

# Apply the replacements using applymap to the entire DataFrame
df = df.applymap(replace_strings)

# Save the modified DataFrame back to a CSV file
df.to_csv('componentsModifiedWithoutIS.csv', index=False)

C:\Users\Viki\AppData\Local\Temp\ipykernel_28880\4249652954.py:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(replace_strings)


In [51]:

# Load the dataset from a CSV file
# Replace 'your_file.csv' with the path to your actual file
df = pd.read_csv('componentsModifiedWithoutIS.csv', header=None)

# Flatten the DataFrame into a single list, keeping track of field positions
data = []
for col in df.columns:
    for row in df.index:
        value = df.iloc[row, col]
        if pd.notna(value):  # Exclude empty elements
            data.append((value.strip(), f"Row {row+1}, Column {col+1}"))

# Create a dictionary to count occurrences and track positions
duplicate_tracker = {}
for value, position in data:
    if value in duplicate_tracker:
        duplicate_tracker[value]['count'] += 1
        duplicate_tracker[value]['positions'].append(position)
    else:
        duplicate_tracker[value] = {'count': 1, 'positions': [position]}

# Write results to a text file
with open('outputDuplicatesWithoutIS.txt', 'w') as output_file:
    output_file.write("Duplicates Found:\n")
    for value, info in duplicate_tracker.items():
        if info['count'] > 1:
            output_file.write(
                f"{value}: {info['count']}; Positions: {', '.join(info['positions'])}\n"
            )

print("Duplicate analysis complete. Results saved to 'outputWithoutIS.txt'.")


Duplicate analysis complete. Results saved to 'outputWithoutIS.txt'.


In [52]:
# we define a simple averaging function that does not depend on intervals
def avg(intervals: np.array, features: np.array) -> np.array:
    try:
        return np.average(features, axis=0)
    except:
        return features

# first we align to words with averaging, collapse_function receives a list of functions
dataset.align(text_field_Words, collapse_functions=[avg])

[2024-12-09 12:23:30.898] | Status  | Unify was called ...
[2024-12-09 12:23:30.898] | Success | Unify completed ...
[2024-12-09 12:23:30.898] | Status  | Pre-alignment based on <CMU_MOSI_TimestampedWords> computational sequence started ...
[2024-12-09 12:23:40.142] | Status  | Pre-alignment done for <CMU_MOSI_COVAREP> ...
[2024-12-09 12:23:41.957] | Status  | Pre-alignment done for <CMU_MOSI_Visual_Facet_41> ...
[2024-12-09 12:23:43.425] | Status  | Pre-alignment done for <CMU_MOSI_TimestampedWordVectors> ...
[2024-12-09 12:23:43.522] | Status  | Alignment starting ...


Aligning zhpQhgha_KU:  96%|█████████▌| 650/676 [00:02<00:00, 334.77 Segments/s]


[2024-12-09 12:26:20.496] | Success | Alignment to <CMU_MOSI_TimestampedWords> complete.
[2024-12-09 12:26:20.496] | Status  | Replacing dataset content with aligned computational sequences
[2024-12-09 12:26:20.506] | Success | Initialized empty <CMU_MOSI_TimestampedWords> computational sequence.
[2024-12-09 12:26:20.506] | Status  | Checking the format of the data in <CMU_MOSI_TimestampedWords> computational sequence ...


[2024-12-09 12:26:20.628] | Success | <CMU_MOSI_TimestampedWords> computational sequence data in correct format.
[2024-12-09 12:26:20.628] | Status  | Checking the format of the metadata in <CMU_MOSI_TimestampedWords> computational sequence ...
[2024-12-09 12:26:20.628] | Warning | <CMU_MOSI_TimestampedWords> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:26:20.628] | Success | Initialized empty <CMU_MOSI_TimestampedWordVectors> computational sequence.
[2024-12-09 12:26:20.628] | Status  | Checking the format of the data in <CMU_MOSI_TimestampedWordVectors> computational sequence ...


[2024-12-09 12:26:20.753] | Success | <CMU_MOSI_TimestampedWordVectors> computational sequence data in correct format.
[2024-12-09 12:26:20.753] | Status  | Checking the format of the metadata in <CMU_MOSI_TimestampedWordVectors> computational sequence ...
[2024-12-09 12:26:20.753] | Warning | <CMU_MOSI_TimestampedWordVectors> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:26:20.753] | Success | Initialized empty <CMU_MOSI_Visual_Facet_41> computational sequence.
[2024-12-09 12:26:20.753] | Status  | Checking the format of the data in <CMU_MOSI_Visual_Facet_41> computational sequence ...


[2024-12-09 12:26:20.876] | Success | <CMU_MOSI_Visual_Facet_41> computational sequence data in correct format.
[2024-12-09 12:26:20.876] | Status  | Checking the format of the metadata in <CMU_MOSI_Visual_Facet_41> computational sequence ...
[2024-12-09 12:26:20.876] | Warning | <CMU_MOSI_Visual_Facet_41> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:26:20.876] | Success | Initialized empty <CMU_MOSI_COVAREP> computational sequence.
[2024-12-09 12:26:20.876] | Status  | Checking the format of the data in <CMU_MOSI_COVAREP> computational sequence ...


[2024-12-09 12:26:21.003] | Success | <CMU_MOSI_COVAREP> computational sequence data in correct format.
[2024-12-09 12:26:21.005] | Status  | Checking the format of the metadata in <CMU_MOSI_COVAREP> computational sequence ...
[2024-12-09 12:26:21.005] | Warning | <CMU_MOSI_COVAREP> computational sequence does not have all the required metadata ... continuing 


In [53]:
label_field = 'CMU_MOSI_Opinion_Labels'

# we add and align to lables to obtain labeled segments
# this time we don't apply collapse functions so that the temporal sequences are preserved
label_recipe = {label_field: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "labels", label_field) + '.csd'}
dataset.add_computational_sequences(label_recipe, destination=None)
dataset.align(label_field)

[2024-12-09 12:26:21.127] | Success | Computational sequence read from file C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\labels\CMU_MOSI_Opinion_Labels.csd ...
[2024-12-09 12:26:21.156] | Status  | Checking the integrity of the <Opinion Segment Labels> computational sequence ...
[2024-12-09 12:26:21.156] | Status  | Checking the format of the data in <Opinion Segment Labels> computational sequence ...


[2024-12-09 12:26:21.249] | Success | <Opinion Segment Labels> computational sequence data in correct format.
[2024-12-09 12:26:21.249] | Status  | Checking the format of the metadata in <Opinion Segment Labels> computational sequence ...
[2024-12-09 12:26:21.249] | Warning | <Opinion Segment Labels> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:26:21.249] | Status  | Unify was called ...


[2024-12-09 12:26:21.431] | Success | Unify completed ...
[2024-12-09 12:26:21.434] | Status  | Pre-alignment based on <CMU_MOSI_Opinion_Labels> computational sequence started ...
[2024-12-09 12:26:21.639] | Status  | Pre-alignment done for <CMU_MOSI_COVAREP> ...
[2024-12-09 12:26:21.815] | Status  | Pre-alignment done for <CMU_MOSI_Visual_Facet_41> ...
[2024-12-09 12:26:22.099] | Status  | Pre-alignment done for <CMU_MOSI_TimestampedWordVectors> ...
[2024-12-09 12:26:22.283] | Status  | Pre-alignment done for <CMU_MOSI_TimestampedWords> ...
[2024-12-09 12:26:22.288] | Status  | Alignment starting ...


Aligning 03bSnISJMiM:   0%|          | 0/13 [00:00<?, ? Segments/s]
                                                                   
Aligning 0h-zjBukYpk:   0%|          | 0/25 [00:00<?, ? Segments/s]
                                                                   
Aligning 1iG0909rllw:   0%|          | 0/30 [00:00<?, ? Segments/s]
                                                                   
Aligning 2WGyTLYerpo:  60%|██████    | 38/63 [00:00<00:00, 378.33 Segments/s]
                                                                             
Aligning 5W7Z1C_fDaE:   0%|          | 0/24 [00:00<?, ? Segments/s]
                                                                   
Aligning 6Egk_28TtTM:   0%|          | 0/12 [00:00<?, ? Segments/s]
                                                                   
Aligning 73jzhE8R1TQ:   0%|          | 0/19 [00:00<?, ? Segments/s]
                                                                   
Aligning 8OtFthrtaJM:   0%| 

[2024-12-09 12:26:27.251] | Success | Alignment to <CMU_MOSI_Opinion_Labels> complete.
[2024-12-09 12:26:27.251] | Status  | Replacing dataset content with aligned computational sequences
[2024-12-09 12:26:27.518] | Success | Initialized empty <CMU_MOSI_TimestampedWords> computational sequence.
[2024-12-09 12:26:27.518] | Status  | Checking the format of the data in <CMU_MOSI_TimestampedWords> computational sequence ...


[2024-12-09 12:26:27.530] | Success | <CMU_MOSI_TimestampedWords> computational sequence data in correct format.
[2024-12-09 12:26:27.530] | Status  | Checking the format of the metadata in <CMU_MOSI_TimestampedWords> computational sequence ...
[2024-12-09 12:26:27.530] | Warning | <CMU_MOSI_TimestampedWords> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:26:27.530] | Success | Initialized empty <CMU_MOSI_TimestampedWordVectors> computational sequence.
[2024-12-09 12:26:27.530] | Status  | Checking the format of the data in <CMU_MOSI_TimestampedWordVectors> computational sequence ...


[2024-12-09 12:26:27.543] | Success | <CMU_MOSI_TimestampedWordVectors> computational sequence data in correct format.
[2024-12-09 12:26:27.543] | Status  | Checking the format of the metadata in <CMU_MOSI_TimestampedWordVectors> computational sequence ...
[2024-12-09 12:26:27.543] | Warning | <CMU_MOSI_TimestampedWordVectors> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:26:27.543] | Success | Initialized empty <CMU_MOSI_Visual_Facet_41> computational sequence.
[2024-12-09 12:26:27.543] | Status  | Checking the format of the data in <CMU_MOSI_Visual_Facet_41> computational sequence ...


[2024-12-09 12:26:27.555] | Success | <CMU_MOSI_Visual_Facet_41> computational sequence data in correct format.
[2024-12-09 12:26:27.555] | Status  | Checking the format of the metadata in <CMU_MOSI_Visual_Facet_41> computational sequence ...
[2024-12-09 12:26:27.555] | Warning | <CMU_MOSI_Visual_Facet_41> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:26:27.555] | Success | Initialized empty <CMU_MOSI_COVAREP> computational sequence.
[2024-12-09 12:26:27.555] | Status  | Checking the format of the data in <CMU_MOSI_COVAREP> computational sequence ...


[2024-12-09 12:26:27.564] | Success | <CMU_MOSI_COVAREP> computational sequence data in correct format.
[2024-12-09 12:26:27.564] | Status  | Checking the format of the metadata in <CMU_MOSI_COVAREP> computational sequence ...
[2024-12-09 12:26:27.564] | Warning | <CMU_MOSI_COVAREP> computational sequence does not have all the required metadata ... continuing 
[2024-12-09 12:26:27.564] | Success | Initialized empty <CMU_MOSI_Opinion_Labels> computational sequence.
[2024-12-09 12:26:27.564] | Status  | Checking the format of the data in <CMU_MOSI_Opinion_Labels> computational sequence ...


[2024-12-09 12:26:27.574] | Success | <CMU_MOSI_Opinion_Labels> computational sequence data in correct format.
[2024-12-09 12:26:27.574] | Status  | Checking the format of the metadata in <CMU_MOSI_Opinion_Labels> computational sequence ...
[2024-12-09 12:26:27.574] | Warning | <CMU_MOSI_Opinion_Labels> computational sequence does not have all the required metadata ... continuing 


In [54]:
num_segments = len(dataset[visual_field_Facet41].keys())  # Assuming all fields have the same segment keys
print(f"Number of segments: {num_segments}")


Number of segments: 2198


In [55]:
# Extract unique video IDs
video_ids = set(seg.split("[")[0] for seg in dataset[visual_field_Facet41].keys())

# Count the number of unique video IDs
num_unique_ids = len(video_ids)

# Count segments for each video ID
video_segment_counts = {video_id: sum(seg.startswith(video_id) for seg in dataset[visual_field_Facet41].keys()) for video_id in video_ids}

# Print the total number of unique video IDs
print(f"Total number of unique video IDs: {num_unique_ids}\n")

# Print the segment counts for each video ID
for video_id, count in video_segment_counts.items():
    print(f"Video {video_id}: {count} segments")



Total number of unique video IDs: 93

Video Vj1wYRQjB-o: 9 segments
Video lXPQBPVc5Cw: 30 segments
Video yDtzw_Y-7RU: 24 segments
Video Dg_0XKD0Mf4: 18 segments
Video 6_0THN4chvY: 14 segments
Video Jkswaaud0hk: 20 segments
Video Sqr0AcuoNnk: 22 segments
Video TvyZBvOMOTc: 15 segments
Video cW1FSBF59ik: 24 segments
Video v0zCBqDeKcE: 16 segments
Video POKffnXeBds: 13 segments
Video Clx4VXItLTE: 31 segments
Video OQvJTdtJ2H4: 16 segments
Video f9O3YtZ2VfI: 30 segments
Video 2iD-tVS8NPw: 30 segments
Video 2WGyTLYerpo: 62 segments
Video Bfr499ggo-0: 22 segments
Video 8d-gEyoeBzc: 30 segments
Video bOL9jKpeJRs: 34 segments
Video 5W7Z1C_fDaE: 24 segments
Video d3_k5Xpfmik: 19 segments
Video 9c67fiY0wGQ: 12 segments
Video Ci-AH39fi3Y: 44 segments
Video phBUpBr1hSo: 21 segments
Video 73jzhE8R1TQ: 19 segments
Video BvYR0L6f2Ig: 26 segments
Video 9T9Hf74oK10: 25 segments
Video X3j2zQgwYgE: 11 segments
Video BXuRRbG0Ugk: 31 segments
Video zhpQhgha_KU: 35 segments
Video VCslbP0mgZI: 17 segments
Vi

In [56]:
# obtain the train/dev/test splits - these splits are based on video IDs
train_split = DATASET.standard_folds.standard_train_fold
dev_split = DATASET.standard_folds.standard_valid_fold
test_split = DATASET.standard_folds.standard_test_fold

# inspect the splits: they only contain video IDs
print(f"lengths: train {len(train_split)}, dev {len(dev_split)}, test {len(test_split)}\n")
print(train_split)
print(dev_split)
print(test_split)

lengths: train 52, dev 10, test 31

['2iD-tVS8NPw', '8d-gEyoeBzc', 'Qr1Ca94K55A', 'Ci-AH39fi3Y', '8qrpnFRGt2A', 'Bfr499ggo-0', 'QN9ZIUWUXsY', '9T9Hf74oK10', '7JsX8y1ysxY', '1iG0909rllw', 'Oz06ZWiO20M', 'BioHAh1qJAQ', '9c67fiY0wGQ', 'Iu2PFX3z_1s', 'Nzq88NnDkEk', 'Clx4VXItLTE', '9J25DZhivz8', 'Af8D0E4ZXaw', 'TvyZBvOMOTc', 'W8NXH0Djyww', '8OtFthrtaJM', '0h-zjBukYpk', 'Vj1wYRQjB-o', 'GWuJjcEuzt8', 'BI97DNYfe5I', 'PZ-lDQFboO8', '1DmNV9C1hbY', 'OQvJTdtJ2H4', 'I5y0__X72p0', '9qR7uwkblbs', 'G6GlGvlkxAQ', '6_0THN4chvY', 'Njd1F0vZSm4', 'BvYR0L6f2Ig', '03bSnISJMiM', 'Dg_0XKD0Mf4', '5W7Z1C_fDaE', 'VbQk4H8hgr0', 'G-xst2euQUc', 'MLal-t_vJPM', 'BXuRRbG0Ugk', 'LSi-o-IrDMs', 'Jkswaaud0hk', '2WGyTLYerpo', '6Egk_28TtTM', 'Sqr0AcuoNnk', 'POKffnXeBds', '73jzhE8R1TQ', 'OtBXNcAL_lE', 'HEsqda8_d0Q', 'VCslbP0mgZI', 'IumbAb8q2dM']
['WKA5OygbEKI', 'c5xsKMxpXnc', 'atnd_PF-Lbs', 'bvLlb-M3UXU', 'bOL9jKpeJRs', '_dI--eQ6qVU', 'ZAIRrfG22O0', 'X3j2zQgwYgE', 'aiEXnCPZubE', 'ZUXBRvtny7o']
['tmZoasNr4rU', 'zhpQhgha_KU', '

In [57]:
# Count the number of segments for each split
train_segments = sum(
    seg.split("[")[0] in train_split for seg in dataset[visual_field_Facet41].keys()
)
dev_segments = sum(
    seg.split("[")[0] in dev_split for seg in dataset[visual_field_Facet41].keys()
)
test_segments = sum(
    seg.split("[")[0] in test_split for seg in dataset[visual_field_Facet41].keys()
)

# Print the results
print(f"Number of segments in train data: {train_segments}")
print(f"Number of segments in validation (dev) data: {dev_segments}")
print(f"Number of segments in test data: {test_segments}")


Number of segments in train data: 1283
Number of segments in validation (dev) data: 229
Number of segments in test data: 686


In [58]:

# A sentinel epsilon for safe division, avoiding division by zero
EPS = 1e-8

word2id = defaultdict(lambda: len(word2id))
UNK = word2id['<unk>']
PAD = word2id['<pad>']
# EOS = word2id['<eos>']
# BOS = word2id['<bos>']
# # SEP = word2id['<sep>']
# DUMMY = word2id['<dummy>']

pattern = re.compile('(.*)\[.*\]')

In [59]:
vocab_size = len(word2id)
print(f"Vocabulary size: {vocab_size}")


# Assuming the input features are already defined elsewhere
# _words, _visual_Facet41, _visual_Facet42, _visual_OpenFace1, _acoustic_COVAREP, _acoustic_OpenSmile_EB10, _wordvectors
# Also assuming train_split, dev_split, test_split are already defined

# Placeholders for final train/dev/test dataset
train = []
dev = []
test = []

# Iterate over the segments in the dataset
num_drop = 0  # Counter to track the number of dropped data points

for segment in dataset[label_field].keys():
    # Get the video ID and features
    vid = re.search(pattern, segment).group(1)
    label = dataset[label_field][segment]['features']
    _words = dataset[text_field_Words][segment]['features']

    # Collect all visual and acoustic features
    _visual_Facet41 = dataset[visual_field_Facet41][segment]['features']
    # _visual_Facet42 = dataset[visual_field_Facet42][segment]['features']
    # _visual_OpenFace1 = dataset[visual_field_OpenFace1][segment]['features']

    _acoustic_COVAREP = dataset[acoustic_field_COVAREP][segment]['features']
    # _acoustic_OpenSmile_EB10 = dataset[acoustic_field_OpenSmile_EB10][segment]['features']

    _wordvectors = dataset[text_field_WordVectors][segment]['features']

    # # Check if all modalities have the same number of elements (length of sequence)
    # if not (_words.shape[0] == _visual_Facet41.shape[0] == _visual_Facet42.shape[0] == _visual_OpenFace1.shape[0] == 
    #         _acoustic_COVAREP.shape[0] == _acoustic_OpenSmile_EB10.shape[0] == _wordvectors.shape[0]):
    #     print(f"Error: Inconsistent sequence lengths for segment {vid}")
    #     num_drop += 1
    #     continue  # Skip this segment and continue with the next one


     # Check if all modalities have the same number of elements (length of sequence)
    if not (_words.shape[0] == _visual_Facet41.shape[0] == _acoustic_COVAREP.shape[0]
                    == _wordvectors.shape[0]):
        print(f"Error: Inconsistent sequence lengths for segment {vid}")
        num_drop += 1
        continue  # Skip this segment and continue with the next one

    # Lists to hold the processed data for each modality
    words = []
    visual = []
    acoustic = []
    wordvectors = []

    # Remove speech pauses (um, uhh, etc.)
    for i, word in enumerate(_words):
        if word[0] != b'sp':  # Remove speech pauses
            words.append(word2id[word[0].decode('utf-8')])  # Decode bytes to string and add to words

            # FIGURE OUT HERE WHAT YOU NEED TO DO - how do you work with the vectors?


            # Append visual features (check the shape of each feature)
            visual.append(_visual_Facet41[i, :])  # Facet41
            # visual.append(_visual_Facet42[i, :])  # Facet42
            # visual.append(_visual_OpenFace1[i, :])  # OpenFace1

            # Append acoustic features (check the shape of each feature)
            acoustic.append(_acoustic_COVAREP[i, :])  # COVAREP
            # acoustic.append(_acoustic_OpenSmile_EB10[i, :])  # OpenSmile_EB10

            # combined_acoustic = np.vstack((_acoustic_COVAREP[i, :], _acoustic_OpenSmile_EB10[i, :]))
            # acoustic.append(combined_acoustic)

            # Append word vectors
            wordvectors.append(_wordvectors[i, :])  # Word vectors


    # LOOK AT THE SHAPES

    # Check the shapes of the collected features before converting to numpy arrays
    # print(f"Word vectors shape: {np.asarray(wordvectors).shape}")
    print(f"Words shape: {np.asarray(words).shape}")
    print(f"WordVectors shape: {np.asarray(wordvectors).shape}")
    print(f"Acoustic shape: {np.asarray(acoustic).shape}")

    print(f"Visual shape: {np.asarray(visual).shape}")

    # Convert lists to numpy arrays
    words = np.asarray(words)
    visual = np.asarray(visual)
    acoustic = np.asarray(acoustic)
    wordvectors = np.asarray(wordvectors)

    # Z-normalization for visual modality (across all visual features)
    std_dev_visual = np.std(visual, axis=0, keepdims=True)
    visual = np.nan_to_num((visual - visual.mean(0, keepdims=True)) / (EPS + std_dev_visual))
    visual[:, std_dev_visual.flatten() == 0] = EPS  # Safeguard for zero standard deviation

    # Z-normalization for acoustic modality (across all acoustic features)
    acoustic_mean = np.nanmean(acoustic, axis=0, keepdims=True)
    std_dev_acoustic = np.nanstd(acoustic, axis=0, keepdims=True)
    std_dev_acoustic = np.nan_to_num(std_dev_acoustic)
    std_dev_acoustic[std_dev_acoustic == 0] = EPS  # Safeguard for zero standard deviation

    acoustic = np.nan_to_num((acoustic - acoustic_mean) / (EPS + std_dev_acoustic))

    # Z-normalization for word vectors
    wordvectors_mean = np.nanmean(wordvectors, axis=0, keepdims=True)
    std_dev_wordvectors = np.nanstd(wordvectors, axis=0, keepdims=True)
    std_dev_wordvectors = np.nan_to_num(std_dev_wordvectors)
    std_dev_wordvectors[std_dev_wordvectors == 0] = EPS  # Safeguard for zero standard deviation
    wordvectors = np.nan_to_num((wordvectors - wordvectors_mean) / (EPS + std_dev_wordvectors))

    # plot_hist2(wordvectors, acoustic)

    # Ensure no NaN or Inf values in the data
    if np.any(np.isnan(acoustic)) or np.any(np.isinf(acoustic)):
        print(f"Error in acoustic data for segment {vid}")
    if np.any(np.isnan(visual)) or np.any(np.isinf(visual)):
        print(f"Error in visual data for segment {vid}")
    if np.any(np.isnan(words)) or np.any(np.isinf(words)):
        print(f"Error in wordvectors data for segment {vid}")

    # Add the data to the appropriate split
    if vid in train_split:
        train.append(((words, visual, acoustic, wordvectors), label, segment))
    elif vid in dev_split:
        dev.append(((words, visual, acoustic, wordvectors), label, segment))
    elif vid in test_split:
        test.append(((words, visual, acoustic,wordvectors), label, segment))
    else:
        print(f"Found video that doesn't belong to any splits: {vid}")

# Output number of dropped datapoints
print(f"Total number of {num_drop} datapoints have been dropped.")
# Check how many words are in the vocabulary
vocab_size = len(word2id)
print(f"Vocabulary size: {vocab_size}")


Vocabulary size: 2
Words shape: (5,)
WordVectors shape: (5, 300)
Acoustic shape: (5, 74)
Visual shape: (5, 47)
Words shape: (19,)
WordVectors shape: (19, 300)
Acoustic shape: (19, 74)
Visual shape: (19, 47)
Words shape: (9,)
WordVectors shape: (9, 300)
Acoustic shape: (9, 74)
Visual shape: (9, 47)
Words shape: (5,)
WordVectors shape: (5, 300)
Acoustic shape: (5, 74)
Visual shape: (5, 47)
Words shape: (4,)
WordVectors shape: (4, 300)
Acoustic shape: (4, 74)
Visual shape: (4, 47)
Words shape: (5,)
WordVectors shape: (5, 300)
Acoustic shape: (5, 74)
Visual shape: (5, 47)
Words shape: (10,)
WordVectors shape: (10, 300)
Acoustic shape: (10, 74)
Visual shape: (10, 47)
Words shape: (13,)
WordVectors shape: (13, 300)
Acoustic shape: (13, 74)
Visual shape: (13, 47)
Words shape: (7,)
WordVectors shape: (7, 300)
Acoustic shape: (7, 74)
Visual shape: (7, 47)
Words shape: (4,)
WordVectors shape: (4, 300)
Acoustic shape: (4, 74)
Visual shape: (4, 47)
Words shape: (5,)
WordVectors shape: (5, 300)
Aco

C:\Users\Viki\Documents\Thesis\tryout3\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1882: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
C:\Users\Viki\AppData\Local\Temp\ipykernel_28880\894660123.py:105: RuntimeWarning: invalid value encountered in subtract
  acoustic = np.nan_to_num((acoustic - acoustic_mean) / (EPS + std_dev_acoustic))


Words shape: (25,)
WordVectors shape: (25, 300)
Acoustic shape: (25, 74)
Visual shape: (25, 47)
Words shape: (18,)
WordVectors shape: (18, 300)
Acoustic shape: (18, 74)
Visual shape: (18, 47)
Words shape: (7,)
WordVectors shape: (7, 300)
Acoustic shape: (7, 74)
Visual shape: (7, 47)
Words shape: (5,)
WordVectors shape: (5, 300)
Acoustic shape: (5, 74)
Visual shape: (5, 47)
Words shape: (21,)
WordVectors shape: (21, 300)
Acoustic shape: (21, 74)
Visual shape: (21, 47)
Words shape: (11,)
WordVectors shape: (11, 300)
Acoustic shape: (11, 74)
Visual shape: (11, 47)
Words shape: (18,)
WordVectors shape: (18, 300)
Acoustic shape: (18, 74)
Visual shape: (18, 47)
Words shape: (8,)
WordVectors shape: (8, 300)
Acoustic shape: (8, 74)
Visual shape: (8, 47)
Words shape: (8,)
WordVectors shape: (8, 300)
Acoustic shape: (8, 74)
Visual shape: (8, 47)
Words shape: (9,)
WordVectors shape: (9, 300)
Acoustic shape: (9, 74)
Visual shape: (9, 47)
Words shape: (5,)
WordVectors shape: (5, 300)
Acoustic shape

In [60]:
# Ensure the train dataset has elements before sampling
if len(train) > 0:
    # Randomly sample one processed segment
    sample = random.choice(train)

    # Extract components
    (words, visual, acoustic, wordvectors), label, segment = sample

    # Display the details
    print(f"Segment: {segment}")
    print(f"Label: {label}")
    print(f"Words (sample, first 50 if too long): {words[:50] if len(words) > 50 else words}")
    print(f"Words shape: {words.shape}")
    print(f"Visual shape: {visual.shape}")
    print(f"Acoustic shape: {acoustic.shape}")
    print(f"WordVectors shape: {wordvectors.shape}")
else:
    print("The train dataset is empty. Check the preprocessing loop for issues.")


Segment: G6GlGvlkxAQ[20]
Label: [[0.]]
Words (sample, first 50 if too long): [ 119  129   20   38 1422 1423]
Words shape: (6,)
Visual shape: (6, 47)
Acoustic shape: (6, 74)
WordVectors shape: (6, 300)


In [61]:
# turn off the word2id - define a named function here to allow for pickling
def return_unk():
    return UNK
word2id.default_factory = return_unk

In [62]:
# define a function that loads data from GloVe-like embedding files

# 2196017 is the vocab size of GloVe here.

def load_emb(w2i, path_to_embedding, embedding_size=300, embedding_vocab=2196017, init_emb=None):
    print("Len w2i start:", len(w2i))
    if init_emb is None:
        emb_mat = np.random.randn(len(w2i), embedding_size)
    else:
        emb_mat = init_emb
    f = open(path_to_embedding, 'r', encoding='utf-8', errors='replace')
    found = 0
    for line in tqdm_notebook(f, total=embedding_vocab):
        try:
            content = line.strip().split()
            vector = np.asarray(list(map(lambda x: float(x), content[-300:])))
            word = ' '.join(content[:-300])
            if word in w2i:
                idx = w2i[word]
                emb_mat[idx, :] = vector
                found += 1
        except ValueError as e:
            print(f"Skipping invalid line: {line}")
    
    print("Len w2i end:", len(w2i))

    print(f"Found {found} words in the embedding file.")
    return torch.tensor(emb_mat).float()

# After processing the entire file, it returns the embedding matrix as a PyTorch tensor.



In [63]:
def multi_collate_textual(batch):
    '''
    Collate functions assume batch = [Dataset[i] for i in index_set]
    '''
    # for later use we sort the batch in descending order of length
    batch = sorted(batch, key=lambda x: x[0][0].shape[0], reverse=True)
    
    # get the data out of the batch - use pad sequence util functions from PyTorch to pad things
    labels = torch.cat([torch.from_numpy(sample[1]) for sample in batch], dim=0).float()
    sentences = pad_sequence([torch.LongTensor(sample[0][0]) for sample in batch], padding_value=PAD, batch_first=True)
    # vectors = pad_sequence([torch.FloatTensor(sample[0][3]) for sample in batch], batch_first=True)
    # 
    lengths = torch.LongTensor([sample[0][0].shape[0] for sample in batch])
    return sentences, labels, lengths
        

batch_sz = 56
train_loader = DataLoader(train, shuffle=True, batch_size=batch_sz, collate_fn=multi_collate_textual)
dev_loader = DataLoader(dev, shuffle=False, batch_size=batch_sz*3, collate_fn=multi_collate_textual)
test_loader = DataLoader(test, shuffle=False, batch_size=batch_sz*3, collate_fn=multi_collate_textual)

# let's create a temporary dataloader just to see how the batch looks like
temp_loader = iter(DataLoader(test, shuffle=True, batch_size=8, collate_fn=multi_collate_textual))
batch = next(temp_loader)

# print(batch[0].shape) # word vectors, padded to maxlen
print(batch[0].shape) # textual features


print(batch[1])
print(batch[2]) 




torch.Size([8, 17])
tensor([[ 3.0000],
        [ 0.8000],
        [ 0.6000],
        [-1.0000],
        [-1.0000],
        [ 2.0000],
        [-2.0000],
        [-2.0000]])
tensor([17, 15, 15, 13,  7,  6,  5,  5])


In [64]:
# Check how many words are in the vocabulary
vocab_size = len(word2id)
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 2729


In [65]:
# class SingleEncoderModelText(nn.Module):
#     def __init__(self, dic_size, use_glove, encoder_size, num_layers, lr, hidden_dim, dr, num_categories, output_size, embedding_path=WORD_EMB_PATH):
#         super(SingleEncoderModelText, self).__init__()
#         
#         # Parameters
#         self.dic_size = dic_size
#         self.use_glove = use_glove
#         self.encoder_size = encoder_size
#         self.num_layers = num_layers
#         self.lr = lr
#         self.hidden_dim = hidden_dim
#         self.dr = dr
#         self.num_categories = num_categories
#         self.output_size = output_size
#         
#         # Embedding size is 300 if using GloVe, else defined elsewhere
#         self.embed_dim = 300 if self.use_glove else 128  # Adjust accordingly
#     
#         # Embedding layer
#         self.embedding = nn.Embedding(self.dic_size, self.embed_dim)
#         
#         if self.use_glove:
#             print("Using glove")
#             
# 
#     
#         # GRU layer
#         self.gru = nn.GRU(input_size=self.embed_dim, 
#                           hidden_size=self.hidden_dim, 
#                           num_layers=self.num_layers, 
#                           dropout=self.dr, 
#                           batch_first=True)
#         
#         # Fully connected output layer
#         self.fc1 = nn.Linear(self.hidden_dim, self.num_categories)  
#         self.fc2 = nn.Linear(self.num_categories, self.output_size)
#     # 
#     # # Debugging - before passing data to the model, print out some input examples
#     # # Check if all indices in the input are valid
#     # def check_input_indices(input_tensor, vocab_size):
#     #     if input_tensor.max().item() >= vocab_size:
#     #         print(f"Warning: Input contains indices out of range! Max index: {input_tensor.max().item()} but vocab size is {vocab_size}.")
#     #     else:
#     #         print(f"Input indices are within valid range. Max index: {input_tensor.max().item()}.")
#     # 
#     # # Example usage:
#     # check_input_indices(input_tensor, self.dic_size)
#     
# 
#         
#     def forward(self, x, lengths):
#         batch_size = lengths.size(0)
# 
#         if (x.min() < 0 or x.max() >= self.dic_size):
#             raise ValueError(
#                 f"Input indices out of range! Min index: {x.min()}, Max index: {x.max()}, Vocabulary size: {self.dic_size}"
#             )
#         # Step 1: Embedding layer
#         embedded = self.embedding(x)  # [batch_size, seq_length] -> [batch_size, seq_length, embed_dim]
#         
#         # Step 2: GRU layer
#         gru_out, hidden = self.gru(embedded)  # gru_out: [batch_size, seq_length, hidden_dim], hidden: [num_layers, batch_size, hidden_dim]
#         
#         # Step 3: Use the last hidden state (or apply pooling, e.g., mean pooling)
#         # We'll use the hidden state from the last time step of the last GRU layer
#         last_hidden = hidden[-1]  # Shape: [batch_size, hidden_dim]
#         
#         # Step 4: Pass through fully connected layers
#         fc1_out = self.fc1(last_hidden)  # [batch_size, num_categories]
#         output = self.fc2(fc1_out)  # [batch_size, output_size]
#         
#         return output


In [66]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter

class SingleEncoderModelText(nn.Module):
    def __init__(self, dic_size, use_glove, encoder_size, num_layers, hidden_dim, dr, output_size):
        super(SingleEncoderModelText, self).__init__()
        
        # Parameters
        self.dic_size = dic_size
        self.use_glove = use_glove
        self.encoder_size = encoder_size
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.dr = dr
        self.output_size = output_size
        
        
                
        if self.use_glove:
            print("Using glove")

        self.embed_dim = 300 if self.use_glove else 128 
    
        # Embedding layer
        self.embedding = nn.Embedding(self.dic_size, self.embed_dim)

            
        # GRU layer
        self.gru = nn.GRU(input_size=self.embed_dim, 
                          hidden_size=self.hidden_dim, 
                          num_layers=self.num_layers, 
                          dropout=self.dr, 
                          batch_first=True)
        
        # Fully connected output layers
        # self.fc1 = nn.Linear(self.hidden_dim, self.num_categories)  
        self.fc2 = nn.Linear(self.hidden_dim, self.output_size)
        
        # Tanh activation function to scale outputs between -3 and 3
        self.tanh = nn.Tanh()

    def forward(self, x, lengths):
        batch_size = lengths.size(0)

        if (x.min() < 0 or x.max() >= self.dic_size):
            raise ValueError(
                f"Input indices out of range! Min index: {x.min()}, Max index: {x.max()}, Vocabulary size: {self.dic_size}"
            )
        
        embedded = self.embedding(x)  # [batch_size, seq_length] -> [batch_size, seq_length, embed_dim]
        
        gru_out, hidden = self.gru(embedded)  # gru_out: [batch_size, seq_length, hidden_dim], hidden: [num_layers, batch_size, hidden_dim]
        
        last_hidden = hidden[-1]  # Shape: [batch_size, hidden_dim]
        
        # fc1_out = self.fc1(last_hidden)  # [batch_size, num_categories]
        output = self.fc2(last_hidden)  # [batch_size, output_size]
        
        output = 3 * self.tanh(output)
        
        return output


In [67]:

temp_loader = iter(DataLoader(test, shuffle=True, batch_size=8, collate_fn=multi_collate_textual))
batch = next(temp_loader)

t, y, l = batch
print(f"Input tensor min: {t.min()}, max: {t.max()}")



# Check how many words are in the vocabulary
vocab_size = len(word2id)
print(f"Vocabulary size: {vocab_size}")
# print(f"vocabulary: {list(word2id.keys())}")

Input tensor min: 1, max: 2449
Vocabulary size: 2729


In [68]:
max_value = float('-inf')  # Initialize to negative infinity
# max_value_v = float('-inf')  # Initialize to negative infinity
for batch in DataLoader(test, shuffle=True, batch_size=8, collate_fn=multi_collate_textual):
    t, y, l = batch
    batch_max = t.max().item()  # Find the max in the current batch
    max_value = max(max_value, batch_max)  # Update the overall max
    # batch_max_v = v.max().item()  # Find the max in the current batch
    # max_value_v = max(max_value_v, batch_max_v)  # Update the overall max


print(f"Overall maximum value in the dataset: {max_value}")
# print(f"Overall maximum value in the dataset: {max_value_v}")

Overall maximum value in the dataset: 2728


In [69]:


torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

CUDA = torch.cuda.is_available()
MAX_EPOCH = 1000


dropout = 0.25
output_size = 1
curr_patience = patience = 8
num_trials = 3
grad_clip_value = 1.0
weight_decay = 0.1

dic_size = len(word2id)
print(f"Dictionary size: {dic_size}")

# define some model settings and hyper-parameters
input_sizes = dic_size
hidden_sizes = int(dic_size)
fc1_size = hidden_sizes

if os.path.exists(CACHE_PATH):
    pretrained_emb, word2id = torch.load(CACHE_PATH)
    print(f"Size of vocabulary (word2id) after pretrained 1: {len(word2id)}")

elif WORD_EMB_PATH is not None:
    pretrained_emb = load_emb(word2id, WORD_EMB_PATH)
    torch.save((pretrained_emb, word2id), CACHE_PATH)
    print(f"Size of vocabulary (word2id) after pretrained 2: {len(word2id)}")
    print(list(word2id.keys()))
else:
    pretrained_emb = None
    
print(f"Size of pre-trained embeddings: {pretrained_emb.shape}")
print(f"Size of vocabulary (word2id): {len(word2id)}")

dic_size = len(word2id)
print(f"Dictionary size: {dic_size}")



model = SingleEncoderModelText(dic_size=dic_size,
                                use_glove=True,
                                encoder_size=300,
                                num_layers=2,
                                hidden_dim=128,
                                dr=0.2,
                                output_size=output_size)


if pretrained_emb is not None:
    model.embedding.weight.data = pretrained_emb
model.embedding.requires_grad = False

optimizer = Adam([param for param in model.parameters() if param.requires_grad], weight_decay=weight_decay)

if CUDA:
    model.cuda()
    
criterion = nn.MSELoss(reduction='sum')

best_valid_loss = float('inf')

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
lr_scheduler.step() # for some reason it seems the StepLR needs to be stepped once first

train_losses = []
valid_losses = []
for e in range(MAX_EPOCH):
    model.train()
    train_iter = tqdm_notebook(train_loader)
    train_loss = 0.0
    
    for batch in train_iter:
        model.zero_grad()
        t, y, l = batch
        batch_size = t.size(0)
        if CUDA:
            t = t.cuda()
            y = y.cuda()
            l = l.cuda()
        
        y_tilde = model(t, l)
        # print(f"batch_pred: {y_tilde.shape}")
        # print(f"labels: {y.shape}")
        
        loss = criterion(y_tilde, y)
        loss.backward()
        torch.nn.utils.clip_grad_value_([param for param in model.parameters() if param.requires_grad], grad_clip_value)
        optimizer.step()
        train_iter.set_description(f"Epoch {e}/{MAX_EPOCH}, current batch loss: {round(loss.item()/batch_size, 4)}")
        train_loss += loss.item()
    train_loss = train_loss / len(train)
    train_losses.append(train_loss)
    print(f"Training loss: {round(train_loss, 4)}")
    
    print ("Loop completed")

    model.eval()
    with torch.no_grad():
        valid_loss = 0.0
        for batch in dev_loader:
            model.zero_grad()
            t, y, l = batch
            if CUDA:
                t = t.cuda()
                y = y.cuda()
                l = l.cuda()
            y_tilde = model(t, l)
            loss = criterion(y_tilde, y)
            valid_loss += loss.item()

    valid_loss = valid_loss/len(dev)
    valid_losses.append(valid_loss)
    print(f"Validation loss: {round(valid_loss, 4)}")
    print(f"Current patience: {curr_patience}, current trial: {num_trials}.")
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        print("Found new best model on dev set!")
        torch.save(model.state_dict(), 'modeltext.std')
        torch.save(optimizer.state_dict(), 'optimtext.std')
        curr_patience = patience
    else:
        curr_patience -= 1
        if curr_patience <= -1:
            print("Running out of patience, loading previous best model.")
            num_trials -= 1
            curr_patience = patience
            model.load_state_dict(torch.load('modeltext.std'))
            optimizer.load_state_dict(torch.load('optimtext.std'))
            lr_scheduler.step()
            print(f"Current learning rate: {optimizer.state_dict()['param_groups'][0]['lr']}")

    if num_trials <= 0:
        print("Running out of patience, early stopping.")
        break



Dictionary size: 2729
Size of vocabulary (word2id) after pretrained 1: 2724
Size of pre-trained embeddings: torch.Size([2724, 300])
Size of vocabulary (word2id): 2724
Dictionary size: 2724
Using glove


C:\Users\Viki\AppData\Local\Temp\ipykernel_28880\2866753710.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_emb, word2id = torch.load(CACHE_PATH)
C:\Users\Vi

  0%|          | 0/23 [00:00<?, ?it/s]

Training loss: 2.3179
Loop completed
Validation loss: 2.7162
Current patience: 8, current trial: 3.
Found new best model on dev set!


  0%|          | 0/23 [00:00<?, ?it/s]

...

Training loss: 1.6494
Loop completed
Validation loss: 2.5459
Current patience: 2, current trial: 1.


  0%|          | 0/23 [00:00<?, ?it/s]

Training loss: 1.6325
Loop completed
Validation loss: 2.5469
Current patience: 1, current trial: 1.


  0%|          | 0/23 [00:00<?, ?it/s]

Training loss: 1.6211
Loop completed
Validation loss: 2.547
Current patience: 0, current trial: 1.
Running out of patience, loading previous best model.
Current learning rate: 1.0000000000000002e-06
Running out of patience, early stopping.


In [70]:
# Check how many words are in the vocabulary
vocab_size = len(word2id)
print(f"Vocabulary size: {vocab_size}")
print(f"vocabulary: {list(word2id.keys())}")

Vocabulary size: 2724
vocabulary: ['<unk>', '<pad>', 'anyhow', 'it', 'was', 'really', 'good', 'they', 'didnt', 'do', 'a', 'whole', 'bunch', 'of', 'background', 'info', 'on', 'why', 'she', 'has', 'to', 'fight', 'and', 'be', 'prepared', 'i', 'mean', 'did', 'little', 'bit', 'but', 'not', 'guess', 'live', 'up', 'with', 'more', 'besides', 'that', 'all', 'over', 'pretty', 'there', 'is', 'like', 'someone', 'while', 'lot', 'action', 'oh', 'my', 'god', 'sad', 'part', 'parts', 'awesome', 'its', 'funny', 'now', 'the', 'title', 'movie', 'basically', 'says', 'im', 'even', 'gonna', 'sugar', 'coat', 'this', 'frustrated', 'me', 'such', 'an', 'extreme', 'extent', 'loudly', 'exclaiming', 'at', 'end', 'film', 'reason', 'comic', 'book', 'fan', 'see', 'characters', 'treated', 'responsibly', 'huh', 'before', 'we', 'go', 'must', 'say', 'had', 'surprisingly', 'decent', 'cast', 'strange', 'since', 'one', 'biggest', 'grapes', 'series', 'always', 'hugh', 'jackman', 'best', 'choice', 'play', 'wolverine', 'will', 

In [71]:
# Example: Check the embedding for a specific token index (e.g., index 2)
token_index = 2
embedding_for_token = model.embedding.weight[token_index]
print(f"Embedding for token index {token_index}: {embedding_for_token}")


Embedding for token index 2: tensor([-1.0278e+00, -1.2375e+00, -1.0179e+00, -9.6206e-01, -1.5605e+00,
         3.5322e-01,  1.3199e-01,  3.1876e-01,  3.4129e-01, -5.0854e-01,
        -5.0849e-01, -9.8768e-01,  3.5373e-01, -7.5110e-01, -7.8273e-01,
         1.8694e+00, -2.0730e-02,  8.3606e-02, -7.1699e-02,  1.5613e-01,
        -1.5211e-02,  8.3180e-01,  2.7893e-01, -1.5488e+00, -3.0018e-01,
        -3.5988e-01, -2.9512e-01, -4.3300e-01,  9.2350e-01,  6.9768e-02,
        -4.3994e-01, -1.0730e+00, -6.4519e-01,  2.1363e+00,  6.3903e-01,
        -8.9149e-01,  2.2408e-01,  6.2722e-01, -2.1699e+00,  1.5846e-01,
         2.2621e+00,  2.1559e+00, -1.4944e+00,  1.6351e+00, -9.8304e-01,
        -1.6916e+00,  1.0455e-01,  1.5966e+00, -1.0505e-01,  3.7311e-01,
         3.6885e-01,  3.0262e-01,  1.0454e+00, -8.4131e-01, -2.3543e-02,
        -1.6662e+00, -3.7126e-02,  1.7958e-01, -9.7885e-01,  1.2303e+00,
         4.5627e-01,  5.4771e-01, -3.8954e-01,  4.7174e-01,  9.4754e-01,
         1.3287e+00, -

In [72]:
# model.load_state_dict(torch.load('modeltext.std'))
# y_true = []
# y_pred = []
# criterion_test = nn.MSELoss(reduction='sum')
# 
# model.eval()
# with torch.no_grad():
#     test_loss = 0.0
#     for batch in test_loader:
#         model.zero_grad()
#         t, y, l = batch
#         if CUDA:
#             t = t.cuda()
#             y = y.cuda()
#             l = l.cuda()
#         print(f"Before clamp - Min index: {t.min()}, Max index: {t.max()}")
#         t = torch.clamp(t, min=0, max=dic_size-1)
#         print(f"After clamp - Min index: {t.min()}, Max index: {t.max()}")
# 
#         y_tilde = model(t, l)
#         loss = criterion_test(y_tilde, y)
#         y_true.append(y_tilde.detach().cpu().numpy())
#         y_pred.append(y.detach().cpu().numpy())
#         test_loss += loss.item()
# print(f"Test set performance: {test_loss/len(test)}")
# y_true = np.concatenate(y_true, axis=0)
# y_pred = np.concatenate(y_pred, axis=0)
# 
# y_true_bin = y_true >= 0
# y_pred_bin = y_pred >= 0
# bin_acc = accuracy_score(y_true_bin, y_pred_bin)
# print(f"Test set accuracy is {bin_acc}")

In [73]:
def convert_to_sentiment_category(score):
    if score == 3:
        return 'strongly positive'
    elif score >= 2 and score < 3:
        return 'positive'
    elif score >= 1 and score < 2:
        return 'weakly positive'
    elif score < 1 and score > -1:
        return 'neutral'
    elif score <= -1 and score > -2:
        return 'weakly negative'
    elif score <= -2 and score > -3:
        return 'negative'
    elif score == -3:
        return 'strongly negative'
    else:
        return 'unknown'


In [74]:


# Assume necessary variables like model, test_loader, dic_size, CUDA, etc. are already defined

# Define the loss function (assuming MSELoss as used previously)
criterion_test = nn.MSELoss(reduction='sum')

# Load the model state (assuming it's already saved in 'modeltext.std')
model.load_state_dict(torch.load('modeltext.std'))

# Prepare for evaluation
y_true = []
y_pred = []

# Set model to evaluation mode
model.eval()

# No gradient calculation
with torch.no_grad():
    test_loss = 0.0

    for batch in test_loader:
        model.zero_grad()

        t, y, l = batch

        if CUDA:
            t = t.cuda()
            y = y.cuda()
            l = l.cuda()

        # Check min and max index of input before clamping
        print(f"Before clamp - Min index: {t.min()}, Max index: {t.max()}")
        
        # Clamp the input indices to the valid range
        t = torch.clamp(t, min=0, max=dic_size-1)
        
        # Check min and max index of input after clamping
        print(f"After clamp - Min index: {t.min()}, Max index: {t.max()}")

        # Forward pass
        y_tilde = model(t, l)

        # Check shapes for consistency
        print(f"y_tilde shape: {y_tilde.shape}, y shape: {y.shape}")

        # Compute loss
        loss = criterion_test(y_tilde, y)
        
        # Log the loss for this batch
        print(f"Batch Loss: {loss.item()}")

        y_true.append(y_tilde.detach().cpu().numpy())
        y_pred.append(y.detach().cpu().numpy())

        test_loss += loss.item()

    avg_test_loss = test_loss / len(test_loader)
    print(f"Test set performance (Average Loss): {avg_test_loss}")

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)
    
    y_true_categories = [convert_to_sentiment_category(score) for score in y_true]
    y_pred_categories = [convert_to_sentiment_category(score) for score in y_pred]
    
    print(f"First few values of y_true: {y_true[:10]}")
    print(f"First few true sentiment labels: {y_true_categories[:10]}")
    
    print(f"First few values of y_pred: {y_pred[:10]}")
    print(f"First few predicted sentiment labels: {y_pred_categories[:10]}")
    
    accuracy = accuracy_score(y_true_categories, y_pred_categories)
    print(f"Accuracy: {accuracy}")


Before clamp - Min index: 1, Max index: 2278
After clamp - Min index: 1, Max index: 2278
y_tilde shape: torch.Size([168, 1]), y shape: torch.Size([168, 1])
Batch Loss: 629.4006958007812
Before clamp - Min index: 1, Max index: 2440
After clamp - Min index: 1, Max index: 2440


C:\Users\Viki\AppData\Local\Temp\ipykernel_28880\1472524431.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('modeltext.std'))


y_tilde shape: torch.Size([168, 1]), y shape: torch.Size([168, 1])
Batch Loss: 440.6356201171875
Before clamp - Min index: 1, Max index: 2584
After clamp - Min index: 1, Max index: 2584
y_tilde shape: torch.Size([168, 1]), y shape: torch.Size([168, 1])
Batch Loss: 694.14794921875
Before clamp - Min index: 1, Max index: 2725
After clamp - Min index: 1, Max index: 2723
y_tilde shape: torch.Size([168, 1]), y shape: torch.Size([168, 1])
Batch Loss: 387.1758728027344
Before clamp - Min index: 1, Max index: 2728
After clamp - Min index: 1, Max index: 2723
y_tilde shape: torch.Size([14, 1]), y shape: torch.Size([14, 1])
Batch Loss: 14.89969253540039
Test set performance (Average Loss): 433.2519660949707
First few values of y_true: [[-0.6646251 ]
 [-0.20678532]
 [-0.03896491]
 [-0.02622201]
 [ 0.5347427 ]
 [-1.0966241 ]
 [-0.03164855]
 [-0.40997005]
 [ 1.1333772 ]
 [-1.1008489 ]]
First few true sentiment labels: ['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'weakly negative', 'neutra